# Single Path One-Shot Neural Architecture Search using Random Search

### Make everything a bit faster

Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


# Search the best architecture

## Setup

### Imports

In [1]:
import os
import sys
from typing import Dict

import numpy as np
import torch
import torchvision
from thop import profile
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_val_transform
from supernet import supernet18

### Make everything a bit faster

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [3]:
# Change this value if needed.
batch_size = 512

In [4]:
val_transform = get_val_transform()

test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified


## Load trained supernet

In [5]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [6]:
channel_multipliers = (0.5, 1.0, 2.0)
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05,

In [7]:
checkpoint = torch.load('supernet_base_120.pth', map_location=device)
supernet.load_state_dict(checkpoint)

<All keys matched successfully>

## Apply Random Search

In [8]:
# Utility class to disable unwanted logs.

class suppress_print:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

### Define our small brute-forcer

In [9]:
@torch.no_grad()
def random_search(
        trained_supernet: nn.Module,
        val_dataloader: DataLoader,
        device: torch.device,
        n_architectures_to_test: int,
) -> Dict[str, list]:
    criterion = nn.CrossEntropyLoss()
    model = trained_supernet
    model.eval()

    unique_archs = {(None,)}
    results = {'accuracy': [], 'loss': [], 'macs': [], 'params': [], 'arch': []}
    for i in tqdm(range(n_architectures_to_test)):
        arch = [None]
        while tuple(arch) in unique_archs:
            arch = [np.random.randint(3) for i in range(8)]
        unique_archs.add(tuple(arch))

        model.sample(arch)

        with suppress_print():  # Get rid of nasty logs from thop.
            macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
        results['arch'].append(arch)
        results['macs'].append(macs)
        results['params'].append(params)

        total_loss = 0.0
        total_correct = 0.0
        total_samples = 0
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device=device)
            labels = labels.to(device=device)

            logits = model(inputs)
            loss = criterion(logits, labels) * labels.shape[0]
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        accuracy = total_correct / total_samples
        loss = total_loss / total_samples
        results['accuracy'].append(accuracy)
        results['loss'].append(loss)
        print(
            f'{arch}: '
            f'MACs={(macs / 1e6):.2f}M, '
            f'params={params / 1e6:.2f}M, '
            f'accuracy={accuracy:.2%}, '
            f'loss={loss:.4f}'
        )

    return results

### Run random model evaluation

In [10]:
n_architectures_to_evaluate = 500

random_search_results = random_search(
    supernet,
    test_dataloader,
    device,
    n_architectures_to_evaluate,
)

  0%|▎                                                                                                                                     | 1/500 [00:01<09:48,  1.18s/it]

[2, 2, 2, 2, 0, 1, 2, 1]: MACs=56.73M, params=14.94M, accuracy=85.32%, loss=0.4672


  0%|▌                                                                                                                                     | 2/500 [00:01<06:47,  1.22it/s]

[0, 1, 1, 1, 0, 1, 0, 0]: MACs=28.95M, params=6.57M, accuracy=84.34%, loss=0.4670


  1%|▊                                                                                                                                     | 3/500 [00:02<05:36,  1.48it/s]

...

[0, 0, 2, 2, 1, 1, 2, 1]: MACs=44.30M, params=15.16M, accuracy=83.96%, loss=0.4813


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [03:42<00:00,  2.27it/s]

[0, 2, 2, 0, 0, 1, 1, 0]: MACs=36.64M, params=8.49M, accuracy=84.78%, loss=0.4652


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [03:43<00:00,  2.24it/s]

[2, 1, 1, 1, 0, 1, 2, 2]: MACs=48.45M, params=19.07M, accuracy=84.98%, loss=0.4758


### Save results for further analysis

In [11]:
import json
with open('random_search_500_models_supernet_base_120.json', 'w') as f:
    json.dump(random_search_results, f)